# Catanomics - Computer Vision and pre-processing
The goal of this notebook is to read in an image of a catan board without number or settlements, and draw the board (ports not included) over the image

In [1]:
# Import necessary libraries
import cv2
import numpy as np

## Helper functions

In [2]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
# Rotate the image to find the maximum y_value (straighten the image as much as possible)
def rotate_image(image, angle):
    # Get image size
    height, width = image.shape[:2]
    # Calculate the center of the image
    center = (width / 2, height / 2)
    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

In [4]:
def rotate_point_back(x_prime, y_prime, angle_deg, center):
    """Return the coordinates of a point rotated about a center point

    Keyword arguments:
    x_prime: x-coordinate of a point on a rotated image
    y_prime: y-coordinate of a point on a rotated image
    angle_deg: the degree in which the target image was rotated to get x and y prime
    center: the center of the rotated image
    """
    # Convert angle to radians
    angle_rad = np.radians(angle_deg)
    # Rotate matrix
    rotation_matrix = np.array( [ [np.cos(angle_rad), -np.sin(angle_rad)],
                                  [np.sin(angle_rad), np.cos(angle_rad)] ])
    # Original center
    original_center = np.array(center)

    # Translate points back to origin for rotation
    translated_point = np.array([x_prime, y_prime]) - original_center

    # Apply inverse rotation
    rotated_point = np.dot(rotation_matrix, translated_point)

    # Translate back after rotation
    original_point = rotated_point + original_center

    return int(original_point[0]), int(original_point[1])

## 0. Record images function

In [5]:
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")
# All images will be saved to "sample_imgs"
dir = "./images/sample_imgs"

## 1. Read in the image, detect the border, and find the max/min y point of the permiter

In [6]:
# Read in an image
image = cv2.imread('./images/v2/board00.jpg')
# The input imgs are too big, so reduce to 25%
image = cv2.resize(image, (0,0), fx=.25, fy=.25)
showImage(image)
saveImage("original.jpg", image, dir)

Image saved to ./images/sample_imgs/original.jpg


In [7]:
# Convert the image to hsv
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
showImage(hsv_image)
saveImage("hsv.jpg", hsv_image, dir)

Image saved to ./images/sample_imgs/hsv.jpg


In [8]:
# Define color bounds for the perimiter of the board
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

In [9]:
# Create a binary mask where the blue regions are white, and everything else is black
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
showImage(mask)
saveImage("mask.jpg", mask, dir)

Image saved to ./images/sample_imgs/mask.jpg


In [10]:
# Bitwise AND to keep the blue parts of the image
blue_regions = cv2.bitwise_and(image, image, mask=mask)
showImage(blue_regions, "Blue Regions")
saveImage("blue_regions.jpg", blue_regions, dir)

Image saved to ./images/sample_imgs/blue_regions.jpg


In [11]:
# Perform canny edge detection on the blue_regions

# Convert to grayscale
gray_blue_region = cv2.cvtColor(blue_regions, cv2.COLOR_BGR2GRAY)
# Apply Gaussian Blur
blurred_blue_regions = cv2.GaussianBlur(gray_blue_region, (5,5), 0)
# Canny edge detection
edges = cv2.Canny(blurred_blue_regions, 50, 150)
# Show the result
showImage(edges, "Edges in blue regions")
saveImage("edges.jpg", edges, dir)

Image saved to ./images/sample_imgs/edges.jpg


## 2. Rotate the image 15 to find the upper left and lower right perimiter points
* The rotation ensures that the minimum x value is the upper left perimiter point
* The rotation also ensures that the maximum x value is the lower right perimiter point
* Then, with inverse rotation, we have the upper left and lower right perimiter points 

In [12]:
# Get the rotated images
ul_lr_angle = 15
ul_lr_edges = rotate_image(edges, ul_lr_angle)
ur_ll_angle = -15
ur_ll_edges = rotate_image(edges, ur_ll_angle)
showImage(ul_lr_edges, "Upper left, lower right image")
saveImage("rotated_left.jpg", ul_lr_edges, dir)
showImage(ur_ll_edges, "Upper right, lower left image")
saveImage("rotated_right.jpg", ur_ll_edges, dir)

Image saved to ./images/sample_imgs/rotated_left.jpg
Image saved to ./images/sample_imgs/rotated_right.jpg


## 3. Using the max and min points and the newly rotated images, find all the perimiter points

### 3.1 Calculate bottom and top coordinates

In [13]:
# Find the max and min points in edges

# Get the indices of all nonzero pixels
edge_y, edge_x = np.nonzero(edges)

# Find the max and min y coordinate of the edges
max_y = np.max(edge_y)
min_y = np.min(edge_y)

max_y, min_y

(870, 152)

In [14]:
# Find the x-values where the y is at its max and min
x_values_at_max_y = edge_x[edge_y == max_y]
x_values_at_min_y = edge_x[edge_y == min_y]

# Calculate median x-value where y is at its max and min
center_x_at_max_y = int(np.median(x_values_at_max_y))
center_x_at_min_y = int(np.median(x_values_at_min_y))

print(f"Top point of board: {center_x_at_max_y, max_y}")
print(f"Bottom point of board: {center_x_at_min_y, min_y}")

Top point of board: (377, 870)
Bottom point of board: (382, 152)


In [15]:
# Visualize top and bottom points
cv2.circle(image, (center_x_at_max_y, max_y), 5, (0,0,255), -1)
cv2.circle(image, (center_x_at_min_y, min_y), 5, (0,0,255), -1)
showImage(image)
saveImage("top_and_bottom_points.jpg", image, dir)

Image saved to ./images/sample_imgs/top_and_bottom_points.jpg


In [16]:
# Give better variable names to the points
top_y = min_y
top_x = center_x_at_min_y
bottom_y = max_y
bottom_x = center_x_at_max_y
(top_x, top_y), (bottom_x, bottom_y)

((382, 152), (377, 870))

### 3.2 Calculate upper left and lower right perimiter coordinates

In [17]:
# Find the top left and bottom right perimiter points with the 15 degree rotated image
ul_lr_y_edges, ul_lr_x_edges = np.nonzero(ul_lr_edges)

# Find the max and min x coordinates of the edges
lr_x = np.max(ul_lr_x_edges)
ul_x = np.min(ul_lr_x_edges)

lr_x, ul_x

(731, 32)

In [18]:
# Find the y-values where x is at its max and min
lr_y_values = ul_lr_y_edges[ul_lr_x_edges == lr_x]
ul_y_values = ul_lr_y_edges[ul_lr_x_edges == ul_x]

# Find the median y-value where x is at its max and min
lr_y = int(np.median(lr_y_values))
ul_y = int(np.median(ul_y_values))

print(f"Upper left coordinate in rotated image: {ul_x, ul_y}")
print(f"Lower right coordinate in rotated image: {lr_x, lr_y}")

Upper left coordinate in rotated image: (32, 409)
Lower right coordinate in rotated image: (731, 595)


In [19]:
# Visualize found points
cv2.circle(ul_lr_edges, (ul_x, ul_y), 5, (255, 255, 255), -1)
cv2.circle(ul_lr_edges, (lr_x, lr_y), 5, (255, 255, 255), -1)
showImage(ul_lr_edges)
saveImage("upper_left_lower_right_points.jpg", ul_lr_edges, dir)

Image saved to ./images/sample_imgs/upper_left_lower_right_points.jpg


In [20]:
# Perform inverse rotation on upper left and lower right points

# Get image size
height, width = ul_lr_edges.shape[:2]
# Calculate the center of ul_lr_edges
ul_lr_center = (width / 2, height / 2)

upper_left_x, upper_left_y = rotate_point_back(
                                x_prime = ul_x, 
                                y_prime=ul_y, 
                                angle_deg=ul_lr_angle, 
                                center=ul_lr_center
                             )
lower_right_x, lower_right_y = rotate_point_back(
                                    x_prime=lr_x,
                                    y_prime=lr_y,
                                    angle_deg=ul_lr_angle,
                                    center=ul_lr_center
                               )
# Should be ((68, 322), (695, 683))
(upper_left_x, upper_left_y), (lower_right_x, lower_right_y)

((68, 322), (695, 683))

In [21]:
# Visualize upper left and lower right points on original image
cv2.circle(image, (upper_left_x, upper_left_y), 5, (0, 255, 0), -1)
cv2.circle(image, (lower_right_x, lower_right_y), 5, (255, 0, 0), -1)
showImage(image)
saveImage("ul_lr_on_original.jpg", image, dir)

Image saved to ./images/sample_imgs/ul_lr_on_original.jpg


### 3.3 Calculate upper right and lower left perimiter coordinates

In [22]:
# Find the top left and bottom right perimiter points with the 15 degree rotated image
ur_ll_y_edges, ur_ll_x_edges = np.nonzero(ur_ll_edges)

# Find the max and min x coordinates of the edges
ur_x = np.max(ur_ll_x_edges)
ll_x = np.min(ur_ll_x_edges)

ur_x, ll_x

(722, 25)

In [23]:
# Find the y-values where x is at its max and min
ll_y_values = ur_ll_y_edges[ur_ll_x_edges == ll_x]
ur_y_values = ur_ll_y_edges[ur_ll_x_edges == ur_x]

# Find the median y-value where x is at its max and min
ll_y = int(np.median(ll_y_values))
ur_y = int(np.median(ur_y_values))

print(f"Upper right coordinate in rotated image: {ur_x, ur_y}")
print(f"Lower left coordinates in rotated image: {ll_x, ll_y}")

Upper right coordinate in rotated image: (722, 413)
Lower left coordinates in rotated image: (25, 595)


In [24]:
# Visualize found points
cv2.circle(ur_ll_edges, (ur_x, ur_y), 5, (255, 255, 255), -1)
cv2.circle(ur_ll_edges, (ll_x, ll_y), 5, (255, 255, 255), -1)
showImage(ur_ll_edges) 
saveImage("upper_right_lower_left_points.jpg", ur_ll_edges, dir)

Image saved to ./images/sample_imgs/upper_right_lower_left_points.jpg


In [25]:
# Perform inverse rotation on upper right and lower left points

# Get image size
height, width = ur_ll_edges.shape[:2]
# Calculate the center of ur_ll_edges
ur_ll_center = (width / 2, height / 2)

upper_right_x, upper_right_y = rotate_point_back(
                                  x_prime = ur_x,
                                  y_prime=ur_y,
                                  angle_deg = ur_ll_angle,
                                  center = ur_ll_center
                               )
lower_left_x, lower_left_y = rotate_point_back(
                                x_prime = ll_x,
                                y_prime = ll_y,
                                angle_deg = ur_ll_angle,
                                center = ur_ll_center
                             )
(upper_right_x, upper_right_y), (lower_left_x, lower_left_y)

((686, 327), (60, 683))

In [26]:
# Visualize upper right and lower left points on original image
cv2.circle(image, (upper_right_x, upper_right_y), 5, (0, 255, 0), -1)
cv2.circle(image, (lower_left_x, lower_left_y), 5, (255, 0, 0), -1)
showImage(image)
saveImage("ur_ll_on_original.jpg", image, dir)

Image saved to ./images/sample_imgs/ur_ll_on_original.jpg


# 4. Connecting the perimiter of the board!

In [27]:
# print the coordinates of the perimiter points with NO ROTATION
print(f"Top coordinates: {top_x, top_y}")
print(f"Upper right coordinates: {upper_right_x, upper_right_y}")
print(f"Lower right coordinates: {lower_right_x, lower_right_y}")
print(f"Bottom coordinates: {bottom_x, bottom_y}")
print(f"Lower left coordinates: {lower_left_x, lower_left_y}")
print(f"Upper left coordinates: {upper_left_x, upper_left_y}")

Top coordinates: (382, 152)
Upper right coordinates: (686, 327)
Lower right coordinates: (695, 683)
Bottom coordinates: (377, 870)
Lower left coordinates: (60, 683)
Upper left coordinates: (68, 322)


In [28]:
# draw the perimiter of the board
cv2.line(image, (top_x, top_y), (upper_right_x, upper_right_y), (0,0,255), 5)
cv2.line(image, (upper_right_x, upper_right_y), (lower_right_x, lower_right_y), (0,0,255), 5)
cv2.line(image, (lower_right_x, lower_right_y), (bottom_x, bottom_y), (0,0,255), 5)
cv2.line(image, (bottom_x, bottom_y), (lower_left_x, lower_left_y), (0,0,255), 5)
cv2.line(image, (lower_left_x, lower_left_y), (upper_left_x, upper_left_y), (0,0,255), 5)
cv2.line(image, (upper_left_x, upper_left_y), (top_x, top_y), (0,0,255), 5)
showImage(image)
saveImage("full_perimiter.jpg", image, dir)


Image saved to ./images/sample_imgs/full_perimiter.jpg


In [29]:
# Save the image 
directory_path = "./images/perimiter"
file_name = "perimiter00.jpg"
full_path = f"{directory_path}/{file_name}"
cv2.imwrite(full_path, image)
print(f"Image saved to {full_path}")

Image saved to ./images/perimiter/perimiter00.jpg
